<a href="https://colab.research.google.com/github/carive47/wseeds_cafe/blob/main/ETL_POC/poc_digit_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [1]:

# Instalación de paquetes necesarios
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 googleapiclient xlsxwriter unidecode
!pip install unidecode
!pip install unidecode pydrive
!pip install xlsxwriter

ERROR: Could not find a version that satisfies the requirement googleapiclient (from versions: none)
ERROR: No matching distribution found for googleapiclient
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.3 MB/s eta 0:00:00


In [2]:
# Conexión con Google Drive
from google.colab import drive
from google.colab import auth
from googleapiclient.discovery import build  # API de Google

# Manipulación y análisis de datos
import pandas as pd
import requests  # Para hacer peticiones HTTP

# Utilidades para comparación y procesamiento de texto
from difflib import get_close_matches  # Encontrar coincidencias aproximadas en texto
import datetime  # Manejo de fechas y tiempos
import re  # Expresiones regulares para limpieza de datos
import os  # Interacción con el sistema de archivos
import unidecode  # Normalización de caracteres especiales en texto
import unidecode
import logging
import uuid

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Acceso a Tablas digitalizadas

In [3]:
# Montar Google Drive
drive.mount('/content/drive')

# Autenticación del usuario para acceso a Drive
auth.authenticate_user()


Mounted at /content/drive


In [4]:
# Conectar con la API de Google Drive
drive_service = build('drive', 'v3')


In [ ]:
# Consulta para obtener carpetas creadas por ti
query = "mimeType='application/vnd.google-apps.folder' and 'root' in parents and trashed=false"

# Obtener lista de archivos (carpetas en este caso)
results = drive_service.files().list(q=query, fields="files(id, name)").execute()

# Mostrar nombres e ID de las carpetas
print("Carpetas creadas por ti:")
for file in results.get("files", []):
    print(f" - Nombre: {file['name']}, ID: {file['id']}")

Carpetas creadas por ti:
 - Nombre: finca_cafe, ID: 1MdUqtY13KIbwZ2wkuVy_YhWMZ45nC-8X
 - Nombre: tablas_unzip, ID: 1NL0FI2-CUQNYzPc43XUZz8tVa-9lXcdj
 - Nombre: Google Earth, ID: 1wvqEviGhGdtes96hnjPA14sujCoYsGs-
 - Nombre: Tablas_Consolidadas_V2, ID: 1jCAyN8YRRJBwlxT4D2lb3EA03WPflB82
 - Nombre: Tablas_Consolidadas, ID: 1JKO3lp0HNBYOvc3GhYTXlbtPt5V_Otza
 - Nombre: mlocal-analytics-python-client-master_deploy, ID: 1JJVC9RYL4qk5-gQvPEqYUYXJhqOrnpWG
 - Nombre: imprimir Abril, ID: 11rN_-21fHbLC3xywdBtBNa90qTxCWULX
 - Nombre: Bitso, ID: 1yOkgz3RpTJjPUfewJjcQb83__GeRB9o6
 - Nombre: Aguablanca, ID: 1bZSNw-_qQzxB648X4vmSrHF8m5U_0l8C
 - Nombre: Colab Notebooks, ID: 1Wb_2tdBTxuyqE09kb3_gdBBem423iLTQ
 - Nombre: CodigosMOOC AlgNeg, ID: 1-bA3KFZVXfdIc_rHnmn-o72_QRPB3l5y
 - Nombre: Classroom, ID: 1ARXecoA6N8J1ebcR8AHpTmAr-R3vHm4yMc9ra73xP0ZXHHHleD5P50T3YXxytKFtIRuJ3wxf
 - Nombre: Uktimas fot, ID: 1Nog8C9JUF0mH9hEdFml5AdBVeDgRFlb6
 - Nombre: celular, ID: 1HThnWshJhrE23Ae4nK4AD3RniuGZnyXZ
 - Nombre: Co

# Descargar tablas

In [ ]:

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Paso 3: Definir ID de carpeta pública de Google Drive
folder_id = '1NL0FI2-CUQNYzPc43XUZz8tVa-9lXcdj'

# Crear carpetas locales para guardar los archivos
local_folder = '/content/datos_finca'
output_folder = '/content/datos_finca_processed'
os.makedirs(local_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

In [ ]:

# Paso 4: Listar y descargar archivos desde la carpeta pública
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
print(f"Descargando {len(file_list)} archivos...")

for file in file_list:
    filename = file['title']
    filepath = os.path.join(local_folder, filename)
    file.GetContentFile(filepath)
    print(f'Descargado: {filename}')

# Paso 5: Procesamiento de archivos descargados
# Define encabezados estándar por actividad
table_headers = {
    "control_maleza": ["LOTE", "FECHA", "METODO", "N.DE LITROS", "VALOR", "N.DE JORNALES", "VALOR", "VALOR TOTAL", "ACUMULADO VALOR"],
    "control_roya": ["LOTES", "FECHA", "PRODUCTO", "P.CARENCIA", "P.INGRESO", "N. DE LITROS", "VALOR", "JORNALES", "VALOR", "VALOR TOTAL", "ACUMULADO VALOR"],
    "fertilizacion": ["LOTES", "FECHA", "TIPO DE ABONO", "DOSIS", "BULTOS", "VALOR", "JORNALES", "VALOR", "VALOR TOTAL", "ACUMULADO"],
    "recoleccion": ["fecha", "kilos café", "N.jornales", "valor.jor", "A. kilos", "A.valor", "A.jor", "bultos", "kilos", "A.bultos", "A.kilos"],
    "renovacion": ["LOTE", "FECHA", "METODO", "N.DE LITROS", "VALOR", "N.DE JORNALES", "VALOR", "VALOR TOTAL", "ACUMULADO VALOR"],
    "deschupon_resiembra": ["LOTES", "FECHA", "VALOR", "ACUMULADO VALOR"]
}
# Diccionario para almacenar las tablas finales por actividad
final_tables = {actividad: [] for actividad in table_headers}

Descargando 41 archivos...
Descargado: primavera__recoleccion.csv
Descargado: la_paloma__control_maleza (2).csv
Descargado: villa_libia__recoleccion (3).csv
Descargado: santa_rita__control_roya.csv
Descargado: villa_libia__recoleccion (2).csv
Descargado: santa_rita__recoleccion.csv
Descargado: primavera__recoleccion (3).csv
Descargado: la_paloma__control_maleza (4).csv
Descargado: santa_rita__recoleccion (1).csv
Descargado: santa_rita__control_maleza (2).csv
Descargado: primavera__control_maleza (6).csv
Descargado: villa_libia__fertilizacion (2).csv
Descargado: primavera__control_maleza (4).csv
Descargado: la_paloma__fertilizacion.csv
Descargado: santa_rita__control_maleza (3).csv
Descargado: santa_rita__fertilizacion.csv
Descargado: villa_libia__control_maleza (3).csv
Descargado: primavera__control_roya.csv
Descargado: primavera__control_maleza (2).csv
Descargado: villa_libia__deschupon_resiembra.csv
Descargado: santa_rita__recoleccion (2).csv
Descargado: villa_libia__control_roya.csv

#consolidar y crear dataframes consolidados

In [ ]:

def process_file(filepath, filename):
    try:
        # Leer el archivo con skipinitialspace=True
        df = pd.read_csv(filepath, delimiter=',', skipinitialspace=True)

        # Eliminar la última columna si está vacía
        if df.columns[-1] == '':
            df = df.iloc[:, :-1]

        # Eliminar filas a partir de "Confidence Scores" en la primera columna
        stop_idx = df[df.iloc[:, 0].astype(str).str.contains("Confidence Scores", na=False)].index
        if not stop_idx.empty:
            df = df.loc[:stop_idx[0]-1]

        # Extract finca and actividad using regex
        match = re.match(r"(.+)__(.+?)(?: \(\d+\))?\.", filename)
        if match:
            finca, actividad = match.groups()
        else:
            print(f" Nombre de archivo inválido: {filename}, se omite el archivo.")
            return

        # Comprobar que el archivo corresponde a una actividad válida
        if actividad not in table_headers:
            print(f" Actividad no válida en {filename}, se omite el archivo.")
            return

        # Seleccionar solo las columnas necesarias
        expected_cols = table_headers[actividad]
        df = df[df.columns[:len(expected_cols)]]  # Seleccionar las primeras n columnas

        # Verificar si el número de columnas es el esperado (después de la selección)
        if len(df.columns) != len(expected_cols):
            print(f" {filename}: columnas esperadas = {len(expected_cols)}, columnas reales = {len(df.columns)} → no se procesará.")
            return

        # Asignar los encabezados correctos
        df.columns = expected_cols

        # Agregar las columnas 'finca' y 'actividad'
        df['finca'] = finca
        df['actividad'] = actividad

        # Agregar la tabla al diccionario de tablas finales
        final_tables[actividad].append(df)

        print(f" Archivo {filename} procesado correctamente.")
    except Exception as e:
        print(f" Error procesando {filename}: {e}")


In [ ]:

# Recorrer los archivos descargados
for file in file_list:
    filename = file['title']
    filepath = os.path.join(local_folder, filename)
    if any(ext in filename for ext in ['.xlsx', '.xls', '.csv']):  # Revisa si contiene las extensiones
        process_file(filepath, filename)

# Unir todas las tablas por actividad y guardarlas
for actividad, tablas in final_tables.items():
    if tablas:  # Solo procesar si hay tablas para esta actividad
        df_final = pd.concat(tablas, ignore_index=True)
        output_filepath = os.path.join(output_folder, f"{actividad}_final.csv")
        df_final.to_csv(output_filepath, index=False)
        print(f"✔️ Tabla final de '{actividad}' guardada con {len(df_final)} filas.")
    else:
        print(f"⚠️ No se encontraron archivos para la actividad {actividad}.")


✔️ Archivo primavera__recoleccion.csv procesado correctamente.
✔️ Archivo la_paloma__control_maleza (2).csv procesado correctamente.
✔️ Archivo villa_libia__recoleccion (3).csv procesado correctamente.
✔️ Archivo santa_rita__control_roya.csv procesado correctamente.
✔️ Archivo villa_libia__recoleccion (2).csv procesado correctamente.
✔️ Archivo santa_rita__recoleccion.csv procesado correctamente.
✔️ Archivo primavera__recoleccion (3).csv procesado correctamente.
✔️ Archivo la_paloma__control_maleza (4).csv procesado correctamente.
✔️ Archivo santa_rita__recoleccion (1).csv procesado correctamente.
✔️ Archivo santa_rita__control_maleza (2).csv procesado correctamente.
✔️ Archivo primavera__control_maleza (6).csv procesado correctamente.
✔️ Archivo villa_libia__fertilizacion (2).csv procesado correctamente.
✔️ Archivo primavera__control_maleza (4).csv procesado correctamente.
✔️ Archivo la_paloma__fertilizacion.csv procesado correctamente.
✔️ Archivo santa_rita__control_maleza (3).csv pr

In [ ]:
for actividad, tablas in final_tables.items():
    if tablas:  # Solo si hay DataFrames para esta actividad
        print(f"\n--- {actividad} ---")  # Imprimir el nombre de la actividad
        for tabla in tablas:  # Iterar sobre los DataFrames de la actividad
            display(tabla)  # Mostrar el DataFrame
    else:
        print(f"⚠️ No se encontraron DataFrames para la actividad {actividad}.")



--- control_maleza ---


,LOTE,FECHA,METODO,N.DE LITROS,VALOR,N.DE JORNALES,VALOR,VALOR TOTAL,ACUMULADO VALOR,finca,actividad
0,'Tomation,'06-27,'Guadaua,',',',','275000,'23'289.400,la_paloma,control_maleza
1,'Batea,'06-27,'C.Q/CO,'3.6+/.5,'126000,'3 cradras,'240.000,'306.000,'23'655.400,la_paloma,control_maleza
2,','06-27,'Guadana,',',',','440.000,'24'095 400,la_paloma,control_maleza
3,'/cenicale,'06-28,'Guadana,',',',','165000,'24'260.400,la_paloma,control_maleza
4,'Ladeva,'07.4,'C.Q/W,'SL+ZL+ZL,'225000,'4er 80000,'320.000,'545000,'24'805.400,la_paloma,control_maleza
5,'carawl,'07.4,'Gradeun,',',',','330000,'25'135 400,la_paloma,control_maleza
6,'culebra,'08.12,'Guadana,',',',','495000,'25'630.400,la_paloma,control_maleza
7,'pubi'2,'07.5,'Guadaia,',',',','880.000,'26'510.400,la_paloma,control_maleza
8,'Guamera,'07-25,'Guadaina,',',',','440.000,'26'950400,la_paloma,control_maleza
9,','07-22,'C.Q/CO,"'9,61+4L",'352000,'Swadras,'640000,'640000,'27'590400,la_paloma,control_maleza


,LOTE,FECHA,METODO,N.DE LITROS,VALOR,N.DE JORNALES,VALOR,VALOR TOTAL,ACUMULADO VALOR,finca,actividad
0,'Batea,'02-2,'Guadana,','450.000 cuadras,'10,'470000,'920.000,'920.000,la_paloma,control_maleza
1,'Motob.,'02.8,"'"" "",'""","'3,5 5",',','350000,'1270.000,NaN,la_paloma,control_maleza
2,'Porven','01.23,'C.Q/W,'201 + 101,'euadras,',','500.000,'1 770.000,la_paloma,control_maleza
3,'culbia,'02-21,'C.Q/CO,','900.000,'20,'940000,'1840000,'3'610.000,la_paloma,control_maleza
4,'fuclera,'02-21,'Guadaina,','Contrato =,"'- 4,5",'- cuadras x 110000,'- 495000,'- 4'105 000,la_paloma,control_maleza
5,'T/ra,'02-21,'C.Q/W,"'2,5+1,5+1.5",'165 000,'contrato,'160.000,'325000,'4'430000,la_paloma,control_maleza
6,'cologouse,'02-23,'C.Q/W,"'52+2,5+2,5",'384000,'11 cuadras,'400.000,'784000,'5'214000,la_paloma,control_maleza
7,'Darubio,'02.29,'Guadana,','contrate,',','220000,'5' 434000,la_paloma,control_maleza
8,'L-5,'03-11,'11,','11,'$110.000/undra,','220000,'5'654000,la_paloma,control_maleza
9,'culebra,'02-29,'C. Q/W,'GLt3L+36,'460 800,'6,'400.000,'860.800,'6'514800,la_paloma,control_maleza


,LOTE,FECHA,METODO,N.DE LITROS,VALOR,N.DE JORNALES,VALOR,VALOR TOTAL,ACUMULADO VALOR,finca,actividad
0,'Tanques,'06 - 11,'Guadaua,',',',','330.000,'7'966.000,santa_rita,control_maleza
1,'Dosbigue,'05-1,'varios,',','55.,','2'640 000,'10'606.000,santa_rita,control_maleza
2,'varior,'06-10,'plateos,',','80,','3'840000,'14 446.000,santa_rita,control_maleza
3,'ED01),'06-19,'Derbey. Guadaria,',',',','550000,'14'996.000,santa_rita,control_maleza
4,'Angrelling,'06 - 24,'11,',',',','330000,'15'326.000,santa_rita,control_maleza
5,'colezaria,'06-20,'C.Q/W,"'10,5+5,5",'403 000,'14,'686000,'1089000,'16'415.000,santa_rita,control_maleza
6,'potserop Corona,'07.12,'C.Q/W,"'11+4,0",'394000,'15,'735000,'1'050000,'17'465000,santa_rita,control_maleza
7,'EDO,'07-25,'C.Q/CO,"'7,5L+2,5",'265000,'10,'490000,'755000,'18'220000,santa_rita,control_maleza
8,',','Graddua,',',"'7,5 wad",'100,'825000,'19'045.000,santa_rita,control_maleza
9,'Alliat,'08-2,'C.Q/CO,'61+2,'212000,"'7,0",'343000,'555.000,'19'600.000,santa_rita,control_maleza


,LOTE,FECHA,METODO,N.DE LITROS,VALOR,N.DE JORNALES,VALOR,VALOR TOTAL,ACUMULADO VALOR,finca,actividad
0,'9,'02.27,'Guadaia,','contrato,',','253000,'9511.000,primavera,control_maleza
1,'11,'02-29,'Quadaia,','11,',','440000,'9'951.000,primavera,control_maleza
2,'2,'03-5,'Desbgercada,',','17,','840000,'10'791.000,primavera,control_maleza
3,'1,'03-5,'plated,',',"'17,5",','840.000,'11'631000,primavera,control_maleza
4,'11,'02-28,'plateD,',','24,','1152000,'13'083.000,primavera,control_maleza
5,'1,'03-7,'Guadaiia,',','contrato,','407.000,'13'490000,primavera,control_maleza
6,'11,'03.1,'Guadaia,',',"'"" "",'""",'440000,'13'930000,NaN,primavera,control_maleza
7,'7,'03-,'C.Q/W,'3L+1.5,'140 000,'3cuadra,'240.000,'380.000,'14'310 000,primavera,control_maleza
8,'3,'03-9,'Guadara,"'3L+1,5L",'191.400,'3wadree,'431.400,'431.400,'14'741400,primavera,control_maleza
9,'6,'03-,'C.Q/CO,'7L+3.5Lt,"'3,5",'440.000,'456000,'896000,'15'637400,primavera,control_maleza


...

,fecha,kilos café,N.jornales,valor.jor,A. kilos,A.valor,A.jor,bultos,kilos,A.bultos,A.kilos,finca,actividad
0,'01 - 5,'359,'11,'359000,'359,'359000,'11,'1,'29,'1,'29,santa_rita,recoleccion
1,'01-12,'422,'10,'422000,'781,'781000,'21,'2,'-,'3,'29,santa_rita,recoleccion
2,'01 - 19,'387,'10,'387000,'1108,'1'108000,'31,'2,'-,'5,'29,santa_rita,recoleccion
3,'02 - 9,'516,'16,'516000,'1684,'1'684000,'47,'2,'23,'7,'52,santa_rita,recoleccion
4,'02-16,'464,'10,'464000,'2148,'2'1 48000,'57,'2,'12,'9,'64,santa_rita,recoleccion
5,'02-23,'1020,'20,'1'020 000,'3168,"'3,168000",'77,'5,'-,'14,'04,santa_rita,recoleccion
6,'03.1,'434,'15,'434000,'3602,'3'602000,'92,'2,'6,'16,'70,santa_rita,recoleccion
7,'03-8,'979,'19,'979000,'4581,'4'581000,'111,'4,'35,'20,'105,santa_rita,recoleccion
8,'03-15,'806,'20,'806000,'5387,'5'387000,'131,'4,'-,'24,'105,santa_rita,recoleccion
9,'03 - 22,'751,'15,'751000,'6138,'6'138000,'146,'3,'35,'27,'140,santa_rita,recoleccion



--- renovacion ---


,LOTE,FECHA,METODO,N.DE LITROS,VALOR,N.DE JORNALES,VALOR,VALOR TOTAL,ACUMULADO VALOR,finca,actividad
0,','04-28,'Siembra,"'0,7 x","'2,5m =",'Caturd,','10.590,',la_paloma,renovacion
1,','Eliminada,'=,'4595x,'110 = . .,'505450,',',',la_paloma,renovacion
2,',',',',',',',',',la_paloma,renovacion
3,','Siem,'bra =,'10590,'x 500 =,'5'295000,',',',la_paloma,renovacion
4,',',',',',',',',',la_paloma,renovacion
5,',',',','#,'5'800.450,',',',la_paloma,renovacion
6,',',',',',',',',',la_paloma,renovacion
7,',',',',',',',',',la_paloma,renovacion
8,',',',',',',',',',la_paloma,renovacion
9,',',',',',',',',',la_paloma,renovacion



--- deschupon_resiembra ---


,LOTES,FECHA,VALOR,ACUMULADO VALOR,finca,actividad
0,'2 7 8,'borbon 11-29,'300.000,'300.000,villa_libia,deschupon_resiembra
1,',',',',villa_libia,deschupon_resiembra
2,',',',',villa_libia,deschupon_resiembra
3,',',',',villa_libia,deschupon_resiembra
4,',',',',villa_libia,deschupon_resiembra
5,',',',',villa_libia,deschupon_resiembra
6,NaN,NaN,NaN,NaN,villa_libia,deschupon_resiembra


In [ ]:


# Unir todas las tablas por actividad y guardarlas
final_dfs = {}
for actividad, tablas in final_tables.items():
    if tablas:  # Solo procesar si hay tablas para esta actividad
        df_final = pd.concat(tablas, ignore_index=True)  # Unir los DataFrames
        final_dfs[actividad] = df_final  # Guardar el DataFrame en el diccionario
        output_filepath = os.path.join(output_folder, f"{actividad}_final.csv")
        df_final.to_csv(output_filepath, index=False)
        print(f" Tabla final de '{actividad}' guardada con {len(df_final)} filas.")
    else:
        print(f" No se encontraron archivos para la actividad {actividad}.")


✔️ Tabla final de 'control_maleza' guardada con 306 filas.
✔️ Tabla final de 'control_roya' guardada con 86 filas.
✔️ Tabla final de 'fertilizacion' guardada con 123 filas.
✔️ Tabla final de 'recoleccion' guardada con 277 filas.
✔️ Tabla final de 'renovacion' guardada con 17 filas.
✔️ Tabla final de 'deschupon_resiembra' guardada con 7 filas.


In [ ]:
final_dfs['control_roya']

,LOTES,FECHA,PRODUCTO,P.CARENCIA,P.INGRESO,N. DE LITROS,VALOR,JORNALES,VALOR,VALOR TOTAL,ACUMULADO VALOR,finca,actividad
0,'Varios,'04.1,'voliam E,'04-15,'04-$7,'7.0 L,'2 730000,',',',',santa_rita,control_roya
1,',','Apristar z,'04-15,'04-$7,'10 L,'2897.500,',',',',santa_rita,control_roya
2,',',',',',',',',',',',santa_rita,control_roya
3,',',"'33,5",'cancers (,"'23,5 +10",'(Sexess),',',',',',santa_rita,control_roya
4,',',',',',',',',','10192235,',santa_rita,control_roya
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,',','poterzol,',','7 L,'196000,',',',',la_paloma,control_roya
82,',',',',',',"',",',',',',la_paloma,control_roya
83,',',',',','TOTAL,'15382.750,',',','54340.950,la_paloma,control_roya
84,'08.6,','Amistar,',','6L+2L,'1'796.000,','canical =,'20,'57'096.950,la_paloma,control_roya


In [ ]:

print("Nombre de las tablas y sus columnas:")
for actividad, df in final_dfs.items():
    print(f"\nTabla: {actividad}")
    print("Columnas:", list(df.columns))

Nombre de las tablas y sus columnas:

Tabla: control_maleza
Columnas: ['LOTE', 'FECHA', 'METODO', 'N.DE LITROS', 'VALOR', 'N.DE JORNALES', 'VALOR', 'VALOR TOTAL', 'ACUMULADO VALOR', 'finca', 'actividad']

Tabla: control_roya
Columnas: ['LOTES', 'FECHA', 'PRODUCTO', 'P.CARENCIA', 'P.INGRESO', 'N. DE LITROS', 'VALOR', 'JORNALES', 'VALOR', 'VALOR TOTAL', 'ACUMULADO VALOR', 'finca', 'actividad']

Tabla: fertilizacion
Columnas: ['LOTES', 'FECHA', 'TIPO DE ABONO', 'DOSIS', 'BULTOS', 'VALOR', 'JORNALES', 'VALOR', 'VALOR TOTAL', 'ACUMULADO', 'finca', 'actividad']

Tabla: recoleccion
Columnas: ['fecha', 'kilos café', 'N.jornales', 'valor.jor', 'A. kilos', 'A.valor', 'A.jor', 'bultos', 'kilos', 'A.bultos', 'A.kilos', 'finca', 'actividad']

Tabla: renovacion
Columnas: ['LOTE', 'FECHA', 'METODO', 'N.DE LITROS', 'VALOR', 'N.DE JORNALES', 'VALOR', 'VALOR TOTAL', 'ACUMULADO VALOR', 'finca', 'actividad']

Tabla: deschupon_resiembra
Columnas: ['LOTES', 'FECHA', 'VALOR', 'ACUMULADO VALOR', 'finca', 'act

#Guardar dataframes en Drive

In [ ]:
# Nombre de la carpeta y el archivo Excel
folder_name = 'finca_cafe'
excel_file_name = 'datos_finca.xlsx'

# Crear la carpeta en Google Drive (si no existe)
folder_path = os.path.join('/content/drive/My Drive', folder_name)
os.makedirs(folder_path, exist_ok=True)

# Ruta completa del archivo Excel
excel_file_path = os.path.join(folder_path, excel_file_name)

# Guardar los DataFrames en el archivo Excel
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    for actividad, df in final_dfs.items():
        df.to_excel(writer, sheet_name=actividad, index=False)

print(f"Archivo Excel '{excel_file_name}' creado en la carpeta '{folder_name}' en tu Google Drive.")

Archivo Excel 'datos_finca.xlsx' creado en la carpeta 'finca_cafe' en tu Google Drive.


# Partimos desde el archivo guardado

se traen las tablas y se empieza por eliminar simbolos como "'" de la tabla y a formatear los nombres de las columnas

In [ ]:

# Ruta al archivo Excel
excel_file_path = '/content/drive/My Drive/finca_cafe/datos_finca.xlsx'

# Leer todas las hojas del archivo Excel
xls = pd.ExcelFile(excel_file_path)

# Obtener la lista de hojas, excluyendo "Tabla dinámica 1"
sheet_names = [sheet for sheet in xls.sheet_names if sheet != "Tabla dinámica 1"]

# Crear un diccionario para almacenar los DataFrames
dataframes = {}

for sheet_name in sheet_names:
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
    df = df.applymap(lambda x: x.replace("'", "") if isinstance(x, str) else x)

    # Convertir nombres de columnas, reemplazar caracteres y eliminar tildes
    df.columns = (
        df.columns.str.lower()
        .str.replace(' ', '', regex=True)
        .str.replace('[. ]', '_', regex=True)
        .str.replace('__', '_', regex=True)
        .str.normalize('NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('ascii')
    )

    # Asignar el DataFrame modificado de nuevo al diccionario
    dataframes[sheet_name] = df

<ipython-input-27-57453c6adaa1>:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace("'", "") if isinstance(x, str) else x)
<ipython-input-27-57453c6adaa1>:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace("'", "") if isinstance(x, str) else x)
<ipython-input-27-57453c6adaa1>:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace("'", "") if isinstance(x, str) else x)
<ipython-input-27-57453c6adaa1>:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace("'", "") if isinstance(x, str) else x)
<ipython-input-27-57453c6adaa1>:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace("'", "") if isinstance(x, str) else x)
<ipython-input-27-57

#Limpieza y Transformacion

In [ ]:
for sheet_name, df in dataframes.items():
    print(f"\n--- {sheet_name}: {df.columns} ---")



--- control_maleza: Index(['lote', 'fecha', 'metodo', 'n_de_litros', 'valor', 'n_de_jornales',
       'valor_1', 'valor_total', 'acumulado_valor', 'finca', 'actividad'],
      dtype='object') ---

--- control_roya: Index(['lotes', 'fecha', 'producto', 'p_carencia', 'p_ingreso', 'n_de_litros',
       'valor', 'jornales', 'valor_1', 'valor_total', 'acumulado_valor',
       'finca', 'actividad'],
      dtype='object') ---

--- fertilizacion: Index(['lotes', 'fecha', 'tipo_de_abono', 'dosis', 'bultos', 'valor',
       'jornales', 'valor_1', 'valor_total', 'acumulado', 'finca',
       'actividad'],
      dtype='object') ---

--- recoleccion: Index(['fecha', 'kilos_cafe', 'n_jornales', 'valor_jor', 'a_kilos', 'a_valor',
       'a_jor', 'bultos', 'kilos', 'a_bultos', 'a_kilos', 'finca',
       'actividad'],
      dtype='object') ---

--- renovacion: Index(['lote', 'fecha', 'metodo', 'n_de_litros', 'valor', 'n_de_jornales',
       'valor_1', 'valor_total', 'acumulado_valor', 'finca', 'activid

In [ ]:
def clean_fecha_column(x):
    try:
        # Buscar el día y el mes usando una expresión regular
        match = re.search(r"(\d+)[./-](\d+)", str(x))
        if match:
            mes = int(match.group(1))
            dia = int(match.group(2))
            # Crear la fecha con el año 2024
            fecha = pd.to_datetime(f"{dia}-{mes}-2024", format="%d-%m-%Y")
            return fecha
        else:
            raise ValueError  # Levantar un error si no se encuentra el patrón
    except ValueError:
        return str(x) + "_formato_erroneo"

def clean_valor_column(x):
    try:
        # Si es una cadena, aplicar replace y convertir a entero
        if isinstance(x, str):
            x = x.replace('.', '').replace(' ', '').replace('-', '')  # Eliminar también '-'
            # Si la cadena está vacía después de eliminar puntos, espacios y '-', convertir a NaN
            if x == '':
                return float('nan')
        # Si no es una cadena y es NaN, devolver NaN
        elif pd.isna(x):  # Verificar si es NaN usando pd.isna()
            return float('nan')
        # Convertir a entero (si es posible)
        return int(x)
    except ValueError:
        # Si hay un error, agregar "_formato_erroneo"
        return str(x) + "_formato_erroneo"


In [ ]:
def limpiar_y_transformar_dataframe(df):
    """Aplica las funciones de limpieza y transformación a un DataFrame."""
    df = df.copy() # Crear una copia explícita del DataFrame

    # Renombrar columnas "lotes" a "lote"
    df.rename(columns={'lotes': 'lote'}, inplace=True)  # Cambio de nombre aquí

    # Eliminar filas vacías
    cols_to_check = [col for col in df.columns if col not in ['finca', 'actividad']]
    df.drop(df[df[cols_to_check].isnull().all(axis=1)].index, inplace=True) # usar drop con inplace=True

    # Formatear columnas con "valor"
    valor_cols = [col for col in df.columns if "valor" in col]
    for col in valor_cols:
        df.loc[:, col] = df[col].apply(clean_valor_column)  # Usar .loc para la asignación

    # Formatear columnas con "fecha"
    fecha_cols = [col for col in df.columns if "fecha" in col]
    for col in fecha_cols:
        df.loc[:, col] = df[col].apply(clean_fecha_column)  # Usar .loc para la asignación

    return df

# Aplicar la función a todos los DataFrames en el diccionario
for sheet_name, df in dataframes.items():
    dataframes[sheet_name] = limpiar_y_transformar_dataframe(df)


In [ ]:
dataframes['control_maleza']

,lote,fecha,metodo,n_delitros,valor,n_dejornales,valor_1,valortotal,acumuladovalor,finca,actividad
0,Tomation,2024-06-27 00:00:00,Guadaua,,NaN,,NaN,275000,23289400,la_paloma,control_maleza
1,Batea,2024-06-27 00:00:00,C.Q/CO,3.6+/.5,126000,3 cradras,240000,306000,23655400,la_paloma,control_maleza
2,,2024-06-27 00:00:00,Guadana,,NaN,,NaN,440000,24095400,la_paloma,control_maleza
3,/cenicale,2024-06-28 00:00:00,Guadana,,NaN,,NaN,165000,24260400,la_paloma,control_maleza
4,Ladeva,2024-07-04 00:00:00,C.Q/W,SL+ZL+ZL,225000,4er 80000,320000,545000,24805400,la_paloma,control_maleza
...,...,...,...,...,...,...,...,...,...,...,...
300,,2024-05-06 00:00:00,C.Q/W,"9,5L+4,5,4,5 contrato",573000,,800000,1373000,21383400,la_paloma,control_maleza
301,Batea,2024-05-30 00:00:00,Guadana,,NaN,,NaN,275000,21658400,la_paloma,control_maleza
302,Danubio,2024-06-20 00:00:00,""" "",""",,NaN,,220000,21878400,NaN,la_paloma,control_maleza
303,Danubio,2024-07-12 00:00:00,plateo,,NaN,17,NaN,816000,22694400,la_paloma,control_maleza


## guardar archivo 1

In [ ]:
excel_file_name2 = 'datos_finca_V2.xlsx'
folder_name = 'finca_cafe'
folder_path = os.path.join('/content/drive/My Drive', folder_name)
excel_file_path = os.path.join(folder_path, excel_file_name2)
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    for sheet_name, df in dataframes.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Archivo Excel '{excel_file_name2}' creado en la carpeta '{folder_name}' en tu Google Drive.")

Archivo Excel 'datos_finca_V2.xlsx' creado en la carpeta 'finca_cafe' en tu Google Drive.


# agrupacion lotes

In [39]:
# Ruta al archivo Excel
excel_file_path = '/content/drive/My Drive/finca_cafe/datos_finca_V2.xlsx'

# Leer todas las hojas del archivo Excel
xls = pd.ExcelFile(excel_file_path)

# Obtener la lista de hojas, excluyendo "Tabla dinámica 1"
sheet_names = [sheet for sheet in xls.sheet_names if sheet != "Tabla dinámica 1"]

# Crear un diccionario para almacenar los DataFrames
dataframes = {}

for sheet_name in sheet_names:
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

    # Asignar el DataFrame modificado de nuevo al diccionario
    dataframes[sheet_name] = df

In [6]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.5 MB/s eta 0:00:00


In [40]:

import unicodedata
from Levenshtein import distance
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Limpieza básica de texto



import re

def limpiar_y_normalizar_lote(df, col_name='lote'):
    """
    Limpia y normaliza la columna de lote:
    - Elimina símbolos que no sean letras/números
    - Elimina letras sueltas (como 'L 5' → '5')
    - Elimina palabras de solo letras cortas/repetidas (como 'ii')
    - Aplica normalización fonética ligera
    """
    def procesar(valor):
        if pd.isna(valor):
            return valor
        # Convertir a string y minúsculas
        valor = str(valor).lower()

        # Eliminar símbolos raros, mantener solo letras, números y espacios
        valor = re.sub(r'[^\w\s]', '', valor)

        # Normalización fonética ligera
        valor = valor.replace('ph', 'f')
        valor = re.sub(r'[b|v]', 'b', valor)
        valor = re.sub(r'[c|k|q]', 'k', valor)
        valor = re.sub(r'[z|s]', 's', valor)
        valor = re.sub(r'[^a-zA-Z0-9\s]', '', valor)  # quitar cosas residuales

        # Separar por espacios y eliminar letras sueltas o repetidas
        partes = valor.strip().split()
        partes = [p for p in partes if not (
            (len(p) == 1 and p.isalpha()) or
            (len(p) <= 2 and p.isalpha()) or
            (len(set(p)) == 1 and len(p) <= 3)  # ej. "ii", "aaa"
        )]

        return " ".join(partes)

    df[col_name] = df[col_name].apply(procesar)
    return df


def expandir_lotes_por_delimitadores(df, col_lote='lote', delimitadores=('-', '.')):
    import re

    # Crear una expresión regular a partir de los delimitadores
    pattern = '|'.join(map(re.escape, delimitadores))

    # Crear una lista para almacenar los nuevos registros
    filas_expandidas = []

    for _, row in df.iterrows():
        lotes = re.split(pattern, str(row[col_lote]))
        lotes = [l.strip() for l in lotes if l.strip()]  # limpiar espacios y vacíos

        for lote in lotes:
            nueva_fila = row.copy()
            nueva_fila[col_lote] = lote
            filas_expandidas.append(nueva_fila)

    df_expandido = pd.DataFrame(filas_expandidas)
    return df_expandido



def clean_text(text):
    text = str(text).lower()
    text = ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def cluster_levenshtein_dbscan(df, col_name='lote_limpio', eps=2, finca=''):
    from Levenshtein import distance
    from sklearn.cluster import DBSCAN

    lotes = df[col_name].tolist()

    # Matriz de distancia modificada con penalización para lotes cortos
    dist_matrix = []
    for a in lotes:
        row = []
        for b in lotes:
            d = distance(a, b)
            if (len(a) <= 2 or len(b) <= 2) and a != b:
                d += 2  # penaliza si ambos son cortos pero diferentes
            row.append(d)
        dist_matrix.append(row)

    clustering = DBSCAN(eps=eps, min_samples=1, metric='precomputed').fit(dist_matrix)
    df['cluster_levenshtein'] = [
        f"{finca}_{label}" if label != -1 else f"{finca}_outlier"
        for label in clustering.labels_
    ]
    return df


# Clustering con Cosine Similarity y AgglomerativeClustering
def cluster_cosine_agglomerative(df, col_name='lote_limpio', n_clusters=5):
    lotes = df[col_name].tolist()
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(lotes)
    similarity_matrix = cosine_similarity(X)
    distance_matrix = 1 - similarity_matrix
    # Este parámetro es 'metric' en versiones nuevas, pero algunas aún requieren 'affinity'
    clustering = AgglomerativeClustering(n_clusters=n_clusters, metric='precomputed', linkage='average')
    df['cluster_cosine'] = clustering.fit_predict(distance_matrix)
    return df

from Levenshtein import distance

def asignar_cluster_por_similitud(df, col_name='lote_limpio', referencia_df=None):
    """Asigna a cada lote el cluster más similar según los lotes ya clusterizados en referencia_df."""

    if referencia_df is None:
        raise ValueError("Debes proporcionar un DataFrame de referencia con columnas 'lote_limpio' y 'cluster_levenshtein'.")

    # Crear diccionario: lote de referencia -> cluster
    ref_lotes = referencia_df[['lote_limpio', 'cluster_levenshtein']].drop_duplicates()
    ref_dict = dict(zip(ref_lotes['lote_limpio'], ref_lotes['cluster_levenshtein']))

    def obtener_cluster_mas_parecido(lote):
        min_dist = float('inf')
        best_cluster = 'sin_match'
        for ref_lote, cluster in ref_dict.items():
            d = distance(str(lote), str(ref_lote))
            if d < min_dist:
                min_dist = d
                best_cluster = cluster
        return best_cluster

    df['cluster_levenshtein'] = df[col_name].apply(obtener_cluster_mas_parecido)
    return df


    df['cluster_levenshtein'] = df[col_name].apply(encontrar_cluster_mas_cercano)
    return df



In [41]:
referencia_maleza = None

for sheet_name in sheet_names:
    df = xls.parse(sheet_name)

    if 'lote' not in df.columns:
        print(f" Hoja '{sheet_name}' no tiene columna 'lote'. Saltando...")
        continue

    df = limpiar_y_normalizar_lote(df, col_name='lote')

    # Expansión de registros por múltiple lote
    df = expandir_lotes_por_delimitadores(df, col_lote='lote', delimitadores=('-', '.'))

    # Limpieza de texto
    df['lote_limpio'] = df['lote'].astype(str).apply(clean_text)

    df_list = []

    if sheet_name == 'control_maleza':
        for finca_name, group in df.groupby('finca'):
            group = cluster_levenshtein_dbscan(group, col_name='lote_limpio', eps=2, finca=finca_name)
            df_list.append(group)

        df_final = pd.concat(df_list, ignore_index=True)
        referencia_maleza = df_final.copy()

    else:
        if referencia_maleza is None:
            print(" No se ha procesado 'control_maleza' todavía. Saltando...")
            continue

        df = asignar_cluster_por_similitud(df, col_name='lote_limpio', referencia_df=referencia_maleza)
        df_list.append(df)
        df_final = pd.concat(df_list, ignore_index=True)

    dataframes[sheet_name] = df_final




 Hoja 'recoleccion' no tiene columna 'lote'. Saltando...


In [42]:
control_maleza = dataframes['control_maleza']
control_maleza

,lote,fecha,metodo,n_delitros,valor,n_dejornales,valor_1,valortotal,acumuladovalor,finca,actividad,lote_limpio,cluster_levenshtein
0,tomation,2024-06-27 00:00:00,Guadaua,NaN,NaN,NaN,NaN,275000,23289400,la_paloma,control_maleza,tomation,la_paloma_0
1,batea,2024-06-27 00:00:00,C.Q/CO,3.6+/.5,126000,3 cradras,240000,306000,23655400,la_paloma,control_maleza,batea,la_paloma_1
2,nan,2024-06-27 00:00:00,Guadana,NaN,NaN,NaN,NaN,440000,24095400,la_paloma,control_maleza,nan,la_paloma_2
3,kenikale,2024-06-28 00:00:00,Guadana,NaN,NaN,NaN,NaN,165000,24260400,la_paloma,control_maleza,kenikale,la_paloma_3
4,ladeba,2024-07-04 00:00:00,C.Q/W,SL+ZL+ZL,225000,4er 80000,320000,545000,24805400,la_paloma,control_maleza,ladeba,la_paloma_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,palma,2024-05-28 00:00:00,"""",NaN,NaN,NaN,NaN,330000,6976000,santa_rita,control_maleza,palma,santa_rita_9
137,korona,06 - 3_formato_erroneo,11,NaN,NaN,NaN,NaN,330000,7306000,santa_rita,control_maleza,korona,santa_rita_17
138,tangeer,2024-06-06 00:00:00,11,NaN,NaN,NaN,NaN,330000,7636000,santa_rita,control_maleza,tangeer,santa_rita_22
139,56,2024-09-06 00:00:00,C.O/co,12205,43500,NaN,104000,147500,15154700,villa_libia,control_maleza,56,villa_libia_0


In [10]:
display(fertilizacion.groupby(['cluster_levenshtein'])['lote'].apply(list).reset_index())


NameError: name 'fertilizacion' is not defined

In [11]:
display(control_maleza.groupby(['cluster_levenshtein'])['lote'].apply(list).reset_index())


,cluster_levenshtein,lote
0,la_paloma_0,[tomation]
1,la_paloma_1,"[batea, batea, batea, batia, batea, batea]"
2,la_paloma_10,"[motob, motob, hotob]"
3,la_paloma_11,"[porben, porton, porton, porton, porton]"
4,la_paloma_12,[fuklera]
5,la_paloma_13,"[tra, tra]"
6,la_paloma_14,[kologouse]
7,la_paloma_15,"[darubio, dambio, danutik, danubio, danubid, d..."
8,la_paloma_16,[l5]
9,la_paloma_17,[petaktory]


In [12]:
for sheet_name, df in dataframes.items():
    print(f"\n--- {sheet_name}: {df.columns} ---")


--- control_maleza: Index(['lote', 'fecha', 'metodo', 'n_delitros', 'valor', 'n_dejornales',
       'valor_1', 'valortotal', 'acumuladovalor', 'finca', 'actividad',
       'lote_limpio', 'cluster_levenshtein'],
      dtype='object') ---

--- control_roya: Index(['lote', 'fecha', 'producto', 'p_carencia', 'p_ingreso', 'n_delitros',
       'valor', 'jornales', 'valor_1', 'valortotal', 'acumuladovalor', 'finca',
       'actividad', 'lote_limpio', 'cluster_levenshtein'],
      dtype='object') ---

--- fertilizacion: Index(['lote', 'fecha', 'tipodeabono', 'dosis', 'bultos', 'valor', 'jornales',
       'valor_1', 'valortotal', 'acumulado', 'finca', 'actividad',
       'lote_limpio', 'cluster_levenshtein'],
      dtype='object') ---

--- recoleccion: Index(['fecha', 'kiloscafe', 'n_jornales', 'valor_jor', 'a_kilos', 'a_valor',
       'a_jor', 'bultos', 'kilos', 'a_bultos', 'a_kilos.1', 'finca',
       'actividad'],
      dtype='object') ---

--- renovacion: Index(['lote', 'fecha', 'metodo', 

# Limpieza 2

In [43]:
def renombrar_columnas_valor_contextual(df):
    """
    Renombra columnas con 'valor' (excepto las que contienen 'total' o 'acumulado')
    para incluir el nombre de la columna anterior, ej: valor_n_dejornales.
    """
    nuevas_columnas = []
    columnas = df.columns.tolist()

    for i, col in enumerate(columnas):
        col_lower = col.lower()
        if 'valor' in col_lower and 'total' not in col_lower and 'acumulado' not in col_lower:
            # Obtener el nombre de la columna anterior si existe
            if i > 0:
                anterior = columnas[i - 1].lower()
                nuevo_nombre = f"valor_{anterior}"
            else:
                nuevo_nombre = col_lower
            nuevas_columnas.append(nuevo_nombre)
        else:
            nuevas_columnas.append(col_lower)

    df.columns = nuevas_columnas
    return df


In [44]:
for sheet_name, df in dataframes.items():
    print(f"Procesando hoja: {sheet_name}")
    df = renombrar_columnas_valor_contextual(df)
    dataframes[sheet_name] = df  # Actualizar en el diccionario


Procesando hoja: control_maleza
Procesando hoja: control_roya
Procesando hoja: fertilizacion
Procesando hoja: recoleccion
Procesando hoja: renovacion
Procesando hoja: deschupon_resiembra


In [15]:
for sheet_name, df in dataframes.items():
    print(f"\n--- {sheet_name}: {df.columns} ---")


--- control_maleza: Index(['lote', 'fecha', 'metodo', 'n_delitros', 'valor_n_delitros',
       'n_dejornales', 'valor_n_dejornales', 'valortotal', 'acumuladovalor',
       'finca', 'actividad', 'lote_limpio', 'cluster_levenshtein'],
      dtype='object') ---

--- control_roya: Index(['lote', 'fecha', 'producto', 'p_carencia', 'p_ingreso', 'n_delitros',
       'valor_n_delitros', 'jornales', 'valor_jornales', 'valortotal',
       'acumuladovalor', 'finca', 'actividad', 'lote_limpio',
       'cluster_levenshtein'],
      dtype='object') ---

--- fertilizacion: Index(['lote', 'fecha', 'tipodeabono', 'dosis', 'bultos', 'valor_bultos',
       'jornales', 'valor_jornales', 'valortotal', 'acumulado', 'finca',
       'actividad', 'lote_limpio', 'cluster_levenshtein'],
      dtype='object') ---

--- recoleccion: Index(['fecha', 'kiloscafe', 'n_jornales', 'valor_n_jornales', 'a_kilos',
       'valor_a_kilos', 'a_jor', 'bultos', 'kilos', 'a_bultos', 'a_kilos.1',
       'finca', 'actividad'],
   

In [45]:
import pandas as pd
import uuid
# 0. Eliminar hora de la fecha
for sheet_name, df in dataframes.items():
    # Identificar columnas que contienen 'fecha' y son de tipo datetime
    fecha_cols = [col for col in df.columns if "fecha" in col.lower()] # No verificamos el tipo aquí

    for col in fecha_cols:
        # Asegurarnos de que no sea ya NaN o una cadena de error
        df[col] = df[col].astype(str).apply(lambda x: x.split(' ')[0] if isinstance(x, str) and ' ' in x and '_formato_erroneo' not in x and x != 'NaT' else x)
        # Convertir de nuevo a datetime, manejando errores
        df[col] = pd.to_datetime(df[col], errors='coerce')

cols_to_dropna = {
    "control_maleza": ["n_delitros","valor_n_delitros","n_dejornales","valor_n_dejornales","valortotal","acumuladovalor"],
    "control_roya": ["producto","p_carencia","p_ingreso","n_delitros","valor_n_delitros","jornales","valor_jornales","valortotal","acumuladovalor"],
    "fertilizacion": ["tipodeabono","dosis","bultos","valor_bultos","jornales","valor_jornales","valortotal","acumulado"],
    "recoleccion": ["kiloscafe","n_jornales","valor_n_jornales","a_kilos","valor_a_kilos","a_jor","bultos","kilos","a_bultos","a_kilos.1"],
    "renovacion": ["metodo","n_delitros","valor_n_delitros","n_dejornales","valor_n_dejornales","valortotal","acumuladovalor"],
    # Asegúrate de que los nombres de las actividades coincidan con tus hojas del Excel
}


# --- Nuevo paso para eliminar filas vacías en columnas específicas ---
for sheet_name, df in dataframes.items():
    if sheet_name in cols_to_dropna:
        cols_to_check = cols_to_dropna[sheet_name]

        # Asegurarse de que las columnas existan en el DataFrame antes de aplicar dropna
        existing_cols_to_check = [col for col in cols_to_check if col in df.columns]


        if existing_cols_to_check:
            initial_rows = len(df)

            # Identificar filas donde *todas* las columnas especificadas son nulas
            # Usamos .loc para evitar SettingWithCopyWarning
            rows_to_drop_mask = df[existing_cols_to_check].isnull().all(axis=1)
            df = df.loc[~rows_to_drop_mask].copy() # Usar .copy() para evitar SettingWithCopyWarning

            rows_dropped = initial_rows - len(df)
            print(f"'{sheet_name}': Eliminadas {rows_dropped} filas donde TODAS las columnas en {existing_cols_to_check} estaban vacías.")
            dataframes[sheet_name] = df # Actualizar el DataFrame en el diccionario

        else:
             print(f"'{sheet_name}': Ninguna de las columnas especificadas para dropna ({cols_to_check}) se encontró en el DataFrame.")
    else:
        print(f"'{sheet_name}': No se especificaron columnas para eliminar filas vacías.")



# Aplicar la función mejorada a todos los DataFrames en el diccionario
for sheet_name, df in dataframes.items():
    # Asegurarse de que la columna 'actividad' existe y tiene el nombre de la hoja
    if 'actividad' not in df.columns:
         df['actividad'] = sheet_name
    dataframes[sheet_name] = asignar_id_unico_mejorado(df, sheet_name[:3])


# 1. Asignar ID único a cada fila
def asignar_id_unico_mejorado(df, prefijo):
    """
    Asigna un ID compuesto a cada fila basado en finca, actividad, cluster, fecha y número de línea.
    Maneja valores NaN para 'cluster_levenshtein'.
    """
    ids = []
    #  columnas necesarias
    required_cols = ['finca', 'actividad', 'fecha']
    for col in required_cols:
        if col not in df.columns:
            print(f"Error: La columna '{col}' no se encuentra en el DataFrame.")
            # Puedes decidir si lanzar un error o manejarlo de otra forma
            return df # O raise ValueError(f"Falta la columna {col}")

    # Asegurarse de que 'cluster_levenshtein' existe, si no, crearla con NaN
    if 'cluster_levenshtein' not in df.columns:
        df['cluster_levenshtein'] = float('nan')

    for index, row in df.iterrows():
        finca = str(row['finca']).replace(' ', '_') # Reemplaza espacios por guiones bajos
        actividad = str(row['actividad']).replace(' ', '_')
        cluster = str(row['cluster_levenshtein']).replace('nan', 'sin_cluster').replace(' ', '_') # Maneja NaN y espacios
        # Formatear la fecha para el ID, si es datetime
        fecha_str = "sin_fecha"
        if pd.notna(row['fecha']):
            try:
                # Intenta formatear como datetime
                fecha_str = row['fecha'].strftime('%Y%m%d')
            except AttributeError:
                # Si no es datetime, intenta limpiarla como string
                fecha_str = re.sub(r'[^\w]', '', str(row['fecha']))[:8] # Limita a 8 caracteres

        linea = index + 1 # Número de línea basado en el índice (empieza en 1)

        # Crea el ID compuesto
        # Usamos un separador consistente como '_'
        id_compuesto = f"{finca}_{actividad}_{cluster}_{fecha_str}_{linea}"
        ids.append(id_compuesto)

    df['id'] = ids
    return df

# Aplicar la función mejorada a todos los DataFrames en el diccionario
for sheet_name, df in dataframes.items():
    # Asegurarse de que la columna 'actividad' existe y tiene el nombre de la hoja
    if 'actividad' not in df.columns:
         df['actividad'] = sheet_name
    dataframes[sheet_name] = asignar_id_unico_mejorado(df, sheet_name[:3])


# 2. Crear tabla de consolidación actividad_general

def crear_actividad_general(dataframes):
    # Definir las columnas de la tabla consolidada
    columnas_consolidadas = ['id', 'finca', 'lote_cluster', 'fecha', 'actividad', 'insumo', 'cantidad_insumo', 'valor_insumo', 'jornales', 'valor_jornales', 'valortotal']

    actividad_general = []
    for sheet_name, df in dataframes.items():
        # Definir lógica para cada hoja (actividad)
        if sheet_name == 'control_maleza':
            data = {
                'id': df['id'],
                'finca': df['finca'],
                'lote_cluster': df['cluster_levenshtein'],
                'fecha': df['fecha'],
                'actividad': sheet_name,
                'insumo': df['metodo'],
                'cantidad_insumo': df['n_delitros'],
                'valor_insumo': df['valor_n_delitros'],
                'jornales': df['n_dejornales'],
                'valor_jornales': df['valor_n_dejornales'],
                'valortotal': df['valortotal']
            }
        elif sheet_name == 'control_roya':
            data = {
                'id': df['id'],
                'finca': df['finca'],
                'lote_cluster': df['cluster_levenshtein'],
                'fecha': df['fecha'],
                'actividad': sheet_name,
                'insumo': df['producto'],
                'cantidad_insumo': df['n_delitros'],
                'valor_insumo': df['valor_n_delitros'],
                'jornales': df['jornales'],
                'valor_jornales': df['valor_jornales'],
                'valortotal': df['valortotal']
            }
        elif sheet_name == 'fertilizacion':
            data = {
                'id': df['id'],
                'finca': df['finca'],
                'lote_cluster': df['cluster_levenshtein'],
                'fecha': df['fecha'],
                'actividad': sheet_name,
                'insumo': df['tipodeabono'],
                'cantidad_insumo': df['bultos'],
                'valor_insumo': df['valor_bultos'],
                'jornales': df['jornales'],
                'valor_jornales': df['valor_jornales'],
                'valortotal': df['valortotal']
            }
        elif sheet_name == 'recoleccion':
            # --- Revisar esta sección cuidadosamente ---
            # Asegurarnos de no referenciar 'lote_cluster' en el df de recoleccion
            data = {
                'id': df['id'],
                'finca': df['finca'],
                'lote_cluster': 'sin_lote',  # Asigna un valor por defecto
                'fecha': df['fecha'],
                'actividad': sheet_name,
                'insumo': df['kiloscafe'],
                'cantidad_insumo': df['kiloscafe'],
                'valor_insumo': 0,
                'jornales': df['n_jornales'],
                'valor_jornales': df['valor_n_jornales'] if 'valor_n_jornales' in df.columns else 0, # Verifica la columna
                'valortotal': df['valortotal'] if 'valortotal' in df.columns else 0 # Verifica la columna
            }
        else:
            continue  # Ignorar otras hojas

        # Crear DataFrame y agregar a la lista
        # Verificar que todas las columnas en columnas_consolidadas estén en data antes de crear el DataFrame
        for col in columnas_consolidadas:
            if col not in data:
                # Esto no debería pasar con el código actual, pero es una buena práctica de depuración
                print(f"Error: Columna '{col}' esperada en '{sheet_name}' pero no encontrada en 'data'.")

        df_subset = pd.DataFrame(data, columns=columnas_consolidadas)
        actividad_general.append(df_subset)

    return pd.concat(actividad_general, ignore_index=True)

actividad_general = crear_actividad_general(dataframes)


'control_maleza': Eliminadas 17 filas donde TODAS las columnas en ['n_delitros', 'valor_n_delitros', 'n_dejornales', 'valor_n_dejornales', 'valortotal', 'acumuladovalor'] estaban vacías.
'control_roya': Eliminadas 17 filas donde TODAS las columnas en ['producto', 'p_carencia', 'p_ingreso', 'n_delitros', 'valor_n_delitros', 'jornales', 'valor_jornales', 'valortotal', 'acumuladovalor'] estaban vacías.
'fertilizacion': Eliminadas 19 filas donde TODAS las columnas en ['tipodeabono', 'dosis', 'bultos', 'valor_bultos', 'jornales', 'valor_jornales', 'valortotal', 'acumulado'] estaban vacías.
'recoleccion': Eliminadas 34 filas donde TODAS las columnas en ['kiloscafe', 'n_jornales', 'valor_n_jornales', 'a_kilos', 'valor_a_kilos', 'a_jor', 'bultos', 'kilos', 'a_bultos', 'a_kilos.1'] estaban vacías.
'renovacion': Eliminadas 12 filas donde TODAS las columnas en ['metodo', 'n_delitros', 'valor_n_delitros', 'n_dejornales', 'valor_n_dejornales', 'valortotal', 'acumuladovalor'] estaban vacías.
'deschu

<ipython-input-45-67cbf8e6d8b3>:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
<ipython-input-45-67cbf8e6d8b3>:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')


In [46]:
actividad_general

,id,finca,lote_cluster,fecha,actividad,insumo,cantidad_insumo,valor_insumo,jornales,valor_jornales,valortotal
0,la_paloma_control_maleza_la_paloma_0_20240627_1,la_paloma,la_paloma_0,2024-06-27,control_maleza,Guadaua,NaN,NaN,NaN,NaN,275000
1,la_paloma_control_maleza_la_paloma_1_20240627_2,la_paloma,la_paloma_1,2024-06-27,control_maleza,C.Q/CO,3.6+/.5,126000,3 cradras,240000,306000
2,la_paloma_control_maleza_la_paloma_2_20240627_3,la_paloma,la_paloma_2,2024-06-27,control_maleza,Guadana,NaN,NaN,NaN,NaN,440000
3,la_paloma_control_maleza_la_paloma_3_20240628_4,la_paloma,la_paloma_3,2024-06-28,control_maleza,Guadana,NaN,NaN,NaN,NaN,165000
4,la_paloma_control_maleza_la_paloma_4_20240704_5,la_paloma,la_paloma_4,2024-07-04,control_maleza,C.Q/W,SL+ZL+ZL,225000,4er 80000,320000,545000
...,...,...,...,...,...,...,...,...,...,...,...
478,santa_rita_recoleccion_sin_cluster_sin_fecha_260,santa_rita,sin_lote,NaT,recoleccion,681,681,0,15,681000,0
479,santa_rita_recoleccion_sin_cluster_sin_fecha_261,santa_rita,sin_lote,NaT,recoleccion,395,395,0,10,395000,0
480,santa_rita_recoleccion_sin_cluster_20240522_262,santa_rita,sin_lote,2024-05-22,recoleccion,356,356,0,10,356000,0
481,santa_rita_recoleccion_sin_cluster_20240628_263,santa_rita,sin_lote,2024-06-28,recoleccion,181,181,0,9,217200,0


In [23]:
actividad_general['id'].duplicated().sum()

np.int64(0)

In [47]:
dataframes['control_maleza']

,lote,fecha,metodo,n_delitros,valor_n_delitros,n_dejornales,valor_n_dejornales,valortotal,acumuladovalor,finca,actividad,lote_limpio,cluster_levenshtein,id
0,tomation,2024-06-27,Guadaua,NaN,NaN,NaN,NaN,275000,23289400,la_paloma,control_maleza,tomation,la_paloma_0,la_paloma_control_maleza_la_paloma_0_20240627_1
1,batea,2024-06-27,C.Q/CO,3.6+/.5,126000,3 cradras,240000,306000,23655400,la_paloma,control_maleza,batea,la_paloma_1,la_paloma_control_maleza_la_paloma_1_20240627_2
2,nan,2024-06-27,Guadana,NaN,NaN,NaN,NaN,440000,24095400,la_paloma,control_maleza,nan,la_paloma_2,la_paloma_control_maleza_la_paloma_2_20240627_3
3,kenikale,2024-06-28,Guadana,NaN,NaN,NaN,NaN,165000,24260400,la_paloma,control_maleza,kenikale,la_paloma_3,la_paloma_control_maleza_la_paloma_3_20240628_4
4,ladeba,2024-07-04,C.Q/W,SL+ZL+ZL,225000,4er 80000,320000,545000,24805400,la_paloma,control_maleza,ladeba,la_paloma_4,la_paloma_control_maleza_la_paloma_4_20240704_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,palma,2024-05-28,"""",NaN,NaN,NaN,NaN,330000,6976000,santa_rita,control_maleza,palma,santa_rita_9,santa_rita_control_maleza_santa_rita_9_2024052...
137,korona,NaT,11,NaN,NaN,NaN,NaN,330000,7306000,santa_rita,control_maleza,korona,santa_rita_17,santa_rita_control_maleza_santa_rita_17_sin_fe...
138,tangeer,2024-06-06,11,NaN,NaN,NaN,NaN,330000,7636000,santa_rita,control_maleza,tangeer,santa_rita_22,santa_rita_control_maleza_santa_rita_22_202406...
139,56,2024-09-06,C.O/co,12205,43500,NaN,104000,147500,15154700,villa_libia,control_maleza,56,villa_libia_0,villa_libia_control_maleza_villa_libia_0_20240...


In [48]:

def crear_tablas_auxiliares(actividad_general):
    # Crear tabla de fincas
    fincas = actividad_general[['finca']].drop_duplicates().reset_index(drop=True)
    fincas['finca_id'] = [f"finca-{uuid.uuid4().hex[:6]}" for _ in range(len(fincas))]
    fincas = fincas.rename(columns={'finca': 'nombre_finca'})  # renombrar aquí directamente

    # Crear tabla de lotes
    lotes = actividad_general[['finca', 'lote_cluster']].drop_duplicates().reset_index(drop=True)
    lotes['lote_id'] = [f"cluster-{uuid.uuid4().hex[:6]}" for _ in range(len(lotes))]

    # Unir con fincas para obtener finca_id
    lotes = lotes.merge(fincas, left_on='finca', right_on='nombre_finca', how='left')
    lotes = lotes[['lote_id', 'lote_cluster', 'finca_id', 'nombre_finca']]
    lotes = lotes.rename(columns={'lote_cluster': 'nombre_lote'})

    return fincas, lotes
fincas, lotes = crear_tablas_auxiliares(actividad_general)

In [ ]:
print(actividad_general.columns)

Index(['id', 'finca', 'lote_cluster', 'fecha', 'actividad', 'insumo',
       'cantidad_insumo', 'valor_insumo', 'jornales', 'valor_jornales',
       'valortotal'],
      dtype='object')


In [ ]:
lotes

,lote_id,nombre_lote,finca_id,nombre_finca
0,cluster-1e65a6,la_paloma_0,finca-dc5a3c,la_paloma
1,cluster-63f6f5,la_paloma_1,finca-dc5a3c,la_paloma
2,cluster-5efb52,la_paloma_2,finca-dc5a3c,la_paloma
3,cluster-c4a70e,la_paloma_3,finca-dc5a3c,la_paloma
4,cluster-068bad,la_paloma_4,finca-dc5a3c,la_paloma
...,...,...,...,...
72,cluster-2134c9,santa_rita_3,finca-61c359,primavera
73,cluster-4eebb0,NaN,finca-61c359,primavera
74,cluster-8657db,NaN,finca-18b0a5,villa_libia
75,cluster-49c109,NaN,finca-3f2f1f,santa_rita


In [ ]:
fincas

,nombre_finca,finca_id
0,la_paloma,finca-dc5a3c
1,primavera,finca-61c359
2,santa_rita,finca-3f2f1f
3,villa_libia,finca-18b0a5


In [49]:
excel_file_name = 'datos_finca_DB_V3.xlsx'

# Define la ruta de la carpeta en Google Drive
folder_name = 'finca_cafe'
folder_path = os.path.join('/content/drive/My Drive', folder_name)
excel_file_path = os.path.join(folder_path, excel_file_name)


# Crea un diccionario que contiene todos los DataFrames
all_dataframes = {
    **dataframes,  # Desempaqueta el diccionario 'dataframes'
    'actividad_general': actividad_general,
    'fincas': fincas,
    'lotes': lotes
}

# Guarda los DataFrames en el archivo Excel
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    for sheet_name, df in all_dataframes.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Archivo Excel '{excel_file_name}' creado en la carpeta '{folder_name}' en tu Google Drive.")

Archivo Excel 'datos_finca_DB_V3.xlsx' creado en la carpeta 'finca_cafe' en tu Google Drive.
